Ссылка на бота: https://t.me/shalom_pizza_bot

# Imports

In [1]:
!pip install pytelegrambotapi

In [2]:
import telebot
from telebot import types

import csv
import random

import pandas as pd

Создаем бота в телеграме с помощью @botFather, получаем токен

In [3]:
bot_token = "7263496012:AAHX7VtbkAj1-5-I6olyOKKHNCUBG_HW7uk"
bot = telebot.TeleBot(bot_token)

In [4]:
# создаем три файла для сохранения необходимых нам данных

# файл для хранения заказов на вынос
with open('order_togo.csv', 'w', newline='') as csvfile:
          fieldnames = ['order', 'toppings']
          writer = csv.writer(csvfile)
          writer.writerow(fieldnames)

# файл для хранения заказов в зале
with open('order_inhall.csv', 'w', newline='') as csvfile:
          fieldnames = ['order', 'toppings']
          writer = csv.writer(csvfile)
          writer.writerow(fieldnames)

# файл для хранения отзывов
with open('reviews.csv', 'w', newline='') as csvfile:
          fieldnames = ['rating', 'review']
          writer = csv.writer(csvfile)
          writer.writerow(fieldnames)

In [5]:
# создаем некоторые необходимые текстовые сообщения-ответы бота

text1 = 'Чтобы получить список команд, нажмите "Помощь"'
text2 = 'Чтобы узнать загруженность зала, нажмите "Загруженность"'
text3 = 'Чтобы сделать заказ на вынос, нажмите "На вынос"'
text4 = 'Чтобы сделать заказ в зал, нажмите "В зале"'
text5 = 'Чтобы оставить отзыв, нажмите "Отзыв"'
text6 = 'Чтобы зайти в админ-панель, нажмите "Админ-панель"'

In [6]:
# читаем необходимые файлы: меню пиццы, меню топпингов для пиццы, меню напитков
# файлы имеют такие колонки:
# name (название); ingr (ингридиенты); price (цена)

menu_pizza = pd.read_csv('menu_pizza.csv')
menu_toppings = pd.read_csv('menu_toppings.csv')
menu_drinks = pd.read_csv('menu_drinks.csv')

In [7]:
# Обработчик команды /start

@bot.message_handler(commands=["start"])
def start(message):

    # Создаем панель с кнопками с возможностью изменения размера
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)

    # Создаем кнопки для панели
    item1 = types.KeyboardButton("Помощь")
    item2 = types.KeyboardButton("Загруженность")
    item3 = types.KeyboardButton("На вынос")
    item4 = types.KeyboardButton("В зале")
    item5 = types.KeyboardButton("Отзыв")
    item6 = types.KeyboardButton("Админ-панель")

    # Располагаем кнопки по рядам
    markup.row(item1)
    markup.row(item3, item4, item5)
    markup.row(item2)
    markup.row(item6)

    # Отправляем сообщение пользователю с прикрепленной панелью
    bot.send_message(message.chat.id, 'Добрый день! Это бот пиццерии Shalom Pizza! :) \n Чем можем помочь?', reply_markup=markup)

In [8]:
# Обработчик сообщений с текстовым содержимым
@bot.message_handler(content_types=["text"])
def handle_text(message):

    # Обработка сообщения "Помощь"
    if message.text.strip() == 'Помощь':
        # Отправляем сообщения с информацией о возможностях бота
        bot.send_message(message.chat.id, 'Вот список возможностей бота:')
        bot.send_message(message.chat.id, text2)
        bot.send_message(message.chat.id, text3)
        bot.send_message(message.chat.id, text4)
        bot.send_message(message.chat.id, text5)
        bot.send_message(message.chat.id, text6)

    # Обработка сообщения "Загруженность"
    elif message.text.strip() == 'Загруженность':
        # Отправляем информацию о текущей загруженности пиццерии
        answer1 = 'Сейчас загруженность пиццерии такая:'
        answer2 = random.choice(['Пусто!', 'Немного людей', 'Аншлаг!'])
        bot.send_message(message.chat.id, answer1)
        bot.send_message(message.chat.id, answer2)

    # Обработка сообщения "На вынос"
    elif message.text.strip() == 'На вынос':
        # Отправляем меню пиццы, напитков и топпингов
        answer1 = 'Отлично! Давайте создадим заказ на вынос. Вот наше меню пиццы:\n'
        for index, row in menu_pizza.iterrows():
            answer1 += f"{row['Name']}, (ингредиенты: {row['Ingr']}), {row['price']}р\n"
        bot.send_message(message.chat.id, answer1)

        answer2 = 'Вот наше меню напитков:\n'
        for index, row in menu_drinks.iterrows():
            answer2 += f"{row['Name']}, {row['price']}р\n"
        bot.send_message(message.chat.id, answer2)

        answer3 = 'Вот наше меню топпингов:\n'
        for index, row in menu_toppings.iterrows():
            answer3 += f"{row['Name']}, {row['price']}р\n"
        bot.send_message(message.chat.id, answer3)

        # Инструкция для пользователя по оформлению заказа
        answer4 = 'Запишите ваш заказ в ответ боту в формате: название_кол-во; топпинги'
        bot.send_message(message.chat.id, answer4)

        # Функция для записи заказа
        def order_writing(message):
            with open('order_togo.csv', 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow([message.text.split(';')[0], message.text.split(';')[1]])
            answer6 = 'Спасибо! Мы получили ваш заказ и уже готовим его!'
            bot.send_message(message.chat.id, answer6)

        # Регистрация следующего шага для получения заказа
        bot.register_next_step_handler(message, order_writing)

    # Обработка сообщения "В зале"
    elif message.text.strip() == 'В зале':
        # Отправляем меню пиццы, напитков и топпингов
        answer1 = 'Отлично! Давайте создадим заказ в зал. Вот наше меню пиццы:\n'
        for index, row in menu_pizza.iterrows():
            answer1 += f"{row['Name']}, (ингредиенты: {row['Ingr']}), {row['price']}р\n"
        bot.send_message(message.chat.id, answer1)

        answer2 = 'Вот наше меню напитков:\n'
        for index, row in menu_drinks.iterrows():
            answer2 += f"{row['Name']}, {row['price']}р\n"
        bot.send_message(message.chat.id, answer2)

        answer3 = 'Вот наше меню топпингов:\n'
        for index, row in menu_toppings.iterrows():
            answer3 += f"{row['Name']}, {row['price']}р\n"
        bot.send_message(message.chat.id, answer3)

        # Инструкция для пользователя по оформлению заказа
        answer4 = 'Запишите ваш заказ в ответ боту в формате: название_кол-во; топпинги'
        bot.send_message(message.chat.id, answer4)

        # Функция для записи заказа
        def order_writing(message):
            with open('order_inhall.csv', 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow([message.text.split(';')[0], message.text.split(';')[1]])
            answer6 = 'Спасибо! Мы получили ваш заказ и уже готовим его!'
            bot.send_message(message.chat.id, answer6)

        # Регистрация следующего шага для получения заказа
        bot.register_next_step_handler(message, order_writing)

    # Обработка сообщения "Отзыв"
    elif message.text.strip() == 'Отзыв':
        # Инструкция для пользователя по оставлению отзыва
        answer1 = 'Оставьте ваш отзыв в формате: оценка(от 1 до 5); текст_отзыва'
        bot.send_message(message.chat.id, answer1)

        # Функция для записи отзыва
        def review_writing(message):
            with open('reviews.csv', 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow([message.text.split(';')[0], message.text.split(';')[1]])
            answer2 = 'Спасибо!'
            bot.send_message(message.chat.id, answer2)

        # Регистрация следующего шага для получения отзыва
        bot.register_next_step_handler(message, review_writing)

    # Обработка сообщения "Админ-панель"
    elif message.text.strip() == 'Админ-панель':
        # Инструкция для доступа к админ-панели
        answer1 = 'Для доступа к админ-панели введите пароль'
        bot.send_message(message.chat.id, answer1)

        # Функция для обработки ввода пароля и предоставления доступа к админ-панели
        def admin_panel(message):
            if message.text == '1234':
                with open('order_inhall.csv', 'r', newline='') as csvfile:
                    reader = csv.reader(csvfile)
                    next(reader)  # Пропускаем заголовок
                    order_inhall_count = sum(1 for row in reader)
                with open('order_togo.csv', 'r', newline='') as csvfile:
                    reader = csv.reader(csvfile)
                    next(reader)  # Пропускаем заголовок
                    order_togo_count = sum(1 for row in reader)
                answer2 = f"Кол-во заказов в зал: {order_inhall_count},\nКол-во заказов на вынос: {order_togo_count}"
                bot.send_message(message.chat.id, answer2)

        # Регистрация следующего шага для проверки пароля
        bot.register_next_step_handler(message, admin_panel)

    # Обработка неизвестного сообщения
    else:
        bot.send_message(message.chat.id, 'Бот не знает, что нужно ответить. Нажмите "Помощь"')

In [9]:
# Запускаем бот
bot.polling(none_stop=True, interval=0)